In [36]:
import pandas as pd
import seaborn as sns; sns.set()
from IPython.display import display, Math, Latex
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from pylab import *
import tkinter as tk
from tkinter import filedialog
import os
%matplotlib inline

In [49]:
# Function Definitions

def subtractDataframeColumns(a,b,x,y):
    b.index = a.index
    a['Val_Diff'] = a[y] - b[y]
    return a

def fiberLoader(side, *argv):
    if len(argv) == 0: #No path provided
        root = tk.Tk()
        root.lift()
        root.withdraw()
        scirun_dir = filedialog.askdirectory(title='SCIRun Directory') #Select SCIRun directory
    else:
        scirun_dir = argv[0]
    #return scirun_dir
    amp_files = os.listdir(scirun_dir+'/simulations/Monopolar/'+side)
    fiber_names  =[]
    for amp in amp_files:
        amp = amp[0:-10]
        fiber_names.append(amp)
    fiber_names = np.unique(fiber_names) #only the file names
    
    dir_files = os.listdir(scirun_dir)
    pts = []
    edge = []
    for file in dir_files:
        for name in fiber_names:
            filename = file.split('.')
            if name == filename[0] and '.edge' in file:
                edge.append(file)
            elif name == filename[0] and '.pts' in file:
                pts.append(file)
            else:
                pass
    return scirun_dir, pts, edge, fiber_names

def fiberActivation(path, pts, edge, ampVal, side):
    fiberActivation = []
    m = np.loadtxt(path+'/'+edge) #edges
    linePoints = np.loadtxt(path+'/'+pts) #pts
    thresh = 0.015
    
    # Separate into idividual fibers
    lineCount = 0
    lineEnd = [0]
    for i in range(2,len(m)-1):
        if m[i,0] != m[i-1,1]:
            lineCount = lineCount + 1
            lineEnd.append(i + lineCount)
    lineCount = lineCount + 1
    lineEnd.append(len(linePoints))
    print(lineCount)
    aC = ['0','1','2','3','4']
    ampC = ['-1mA','-2mA','-3mA','-4mA','-5mA']
    scaleA = [1,2,3,4,5]
    counter = 0
    for contact in range(0,4):
        for amplitude in range(1,6):
            scale = amplitude
            
            lineA = np.loadtxt(path+'/simulations/Monopolar/'+side+'/'+ampVal+'_amp_'+str(contact)+'.txt') #Fiber Current Values
            lineA = lineA*scale
            ptVal = []
            max_AF = np.zeros(lineCount)
            for l in range(1,lineCount-1): #change first value back to 1!!!
                fiberA = lineA[lineEnd[l-1]:lineEnd[l]]
                AF = np.diff(np.diff(fiberA))
                if len(AF) < 1:
                    AF = [0]
                max_AF[l-1] = max(AF)
                if max(AF) > thresh:
                    data = np.ones(len(fiberA))
                    data = data.tolist()
                    ptVal.extend(data)
                    #ptVal.append(np.ones(len(fiberA)-1))
                    #np.concatenate((ptVal,np.ones(len(fiberA)-1)), axis=None)
                else:
                    data = np.zeros(len(fiberA))
                    data = data.tolist()
                    ptVal.extend(data)
                    #ptVal.append(np.zeros(len(fiberA)-1))
                    #np.concatenate((ptVal,np.zeros(len(fiberA)-1)), axis=None)
                # Concatenate AF matricies for this fiber bundle
                #at current contact/amp
                if l == 1:
                    AF = [0,AF,0]
                    AFc = AF
                else:
                    AF = [0,AF,0]
                    AFc.append(AF)
            percentActivated = len(np.argwhere(max_AF > thresh))/(lineCount-1)*100
            fiberActivation.append(percentActivated)
            
            lineAmp = lineA
            counter = counter + len(data)
            #print(ampVal, contact, amplitude)
            np.savetxt(path+'/simulations/mono_fiber_activation/'+side.upper()+'/'+ampVal+'/'+str(contact)+'/AF_'+ampVal+'_'+str(contact)+'_-'+str(amplitude)+'mA.txt',ptVal,delimiter=',',fmt='%d')
            #np.savetxt(path+'/simulations/mono_fiber_activation/'+side.upper()+'/'+ampVal+'/'+str(contact)+'/AF_'+ampVal+'_'+str(contact)+'_-'+str(amplitude)+'mA.txt',AFc,delimiter=',',fmt='%0.6f')
    return fiberActivation

#fiberActivation("CL_L.pts","CL_L.edge","CLseed_fiberVoltage_Left_")
#fiberLoader('Left')

# Right Side

In [50]:
# Right data

path, pts, edge, ampVal = fiberLoader('Right') #get pts, edge, and SCIRun output files, Right side

percentActivation_Right = []
for f in range(0,len(ampVal)):
    percentActivation_Right = percentActivation_Right + fiberActivation(path,pts[f],edge[f],ampVal[f],'Right')


df_template_right = pd.read_csv("activation_df_template.csv")
#df_template_right = pd.read_csv("activation_df_template_directional.csv")
#df_template_left.head()
df_right = df_template_right
for f in range(0,len(ampVal)):
    if f > 0:
        #df_left.append(df_template_left)
        df_right = pd.concat([df_right, df_template_right], ignore_index=True)
    for i in range(0,20): #Fiber Names
        df_right.loc[i+(20*f), 'Fiber_Name'] = ampVal[f]

df_right.loc[0:len(percentActivation_Right), 'Percent_Activation'] = percentActivation_Right #values

df_right.to_csv(path+'/simulations/activation_Right.csv', index=False) #Save to csv

df_right.head()

5000
CL_right 0 1
CL_right 0 2
CL_right 0 3
CL_right 0 4
CL_right 0 5
CL_right 1 1
CL_right 1 2
CL_right 1 3
CL_right 1 4
CL_right 1 5
CL_right 2 1
CL_right 2 2
CL_right 2 3
CL_right 2 4
CL_right 2 5
CL_right 3 1
CL_right 3 2
CL_right 3 3
CL_right 3 4
CL_right 3 5
5000
CM_right 0 1
CM_right 0 2
CM_right 0 3
CM_right 0 4
CM_right 0 5
CM_right 1 1
CM_right 1 2
CM_right 1 3
CM_right 1 4
CM_right 1 5
CM_right 2 1
CM_right 2 2
CM_right 2 3
CM_right 2 4
CM_right 2 5
CM_right 3 1
CM_right 3 2
CM_right 3 3
CM_right 3 4
CM_right 3 5
5000
MD_right 0 1
MD_right 0 2
MD_right 0 3
MD_right 0 4
MD_right 0 5
MD_right 1 1
MD_right 1 2
MD_right 1 3
MD_right 1 4
MD_right 1 5
MD_right 2 1
MD_right 2 2
MD_right 2 3
MD_right 2 4
MD_right 2 5
MD_right 3 1
MD_right 3 2
MD_right 3 3
MD_right 3 4
MD_right 3 5
5000
VPL_right 0 1
VPL_right 0 2
VPL_right 0 3
VPL_right 0 4
VPL_right 0 5
VPL_right 1 1
VPL_right 1 2
VPL_right 1 3
VPL_right 1 4
VPL_right 1 5
VPL_right 2 1
VPL_right 2 2
VPL_right 2 3
VPL_right 2 4
VPL_

,Percent_Activation,Contact,Amplitude,Fiber_Name
0,12.722545,0,1,CL_right
1,21.644329,0,2,CL_right
2,29.945989,0,3,CL_right
3,38.207642,0,4,CL_right
4,46.549310,0,5,CL_right


In [ ]:
len(percentActivation_Right)

# Left Side

In [ ]:
# Left data

path, pts, edge, ampVal = fiberLoader('Left',path) # Add path to skip dialog box

percentActivation_Left = []
for f in range(0,len(ampVal)):
    percentActivation_Left = percentActivation_Left + fiberActivation(path,pts[f],edge[f],ampVal[f],'Left')
    
    
df_template_left = pd.read_csv("activation_df_template.csv")
#df_template_left.head()
df_left = df_template_left
for f in range(0,len(ampVal)):
    if f > 0:
        #df_left.append(df_template_left)
        df_left = pd.concat([df_left, df_template_left], ignore_index=True)
    for i in range(0,20): #Fiber Names
        df_left.loc[i+(20*f), 'Fiber_Name'] = ampVal[f]

#df_template_left.loc[0:len(percentActivation_Left), 'Percent_Activation'] = percentActivation_Left #values
df_left.loc[0:len(percentActivation_Left), 'Percent_Activation'] = percentActivation_Left #values

df_left.to_csv(path+'/simulations/activation_Left.csv', index=False) #Save to csv

#df_left = pd.read_csv(path+'/simulations/activation_Left.csv')
df_left.head()

In [ ]:
pd.Series(df_left['Fiber_Name']).unique()

# Complete Plot

In [ ]:
pd.options.mode.chained_assignment = None #hides some annoying warnings (nothing to worry about)
df_left_sub = df_left
df_right_sub = df_right

#Replace fiber names to be more readable
#fibers_old = ['CL_L','CM_L','CM_UF_tracks','MD_L','VPL_L']
fibers_old = pd.Series(df_left['Fiber_Name']).unique()
fibers_new = ['CL','CM','MD','VPL','VPM'] #change these when you need to
count = 0
for fiber_old in fibers_old:
    indices = [i for i, x in enumerate(df_left_sub['Fiber_Name']) if x == fiber_old]
    df_left_sub['Fiber_Name'][indices] = fibers_new[count] #helps set titles in the legend
    df_right_sub['Fiber_Name'][indices] = fibers_new[count]
    count += 1



plt.rcParams['figure.figsize']=20,10
sns.set(style='white',color_codes=True)

#colors = ["#253494", "#F2F275", "#2c7fb8", "#54DB81", "#47D8EA"]
colors = ["#253494", "#54DB81", "#F2F275", "#2c7fb8", "#47D8EA"]
#colors = ['#253494', '#F2F275', '#ffb026', '#d55e00', '#47D8EA']
fig = plt.figure()
location = [8,6,4,2] #subplot locations

for contact in range(0,4,1):
    #print(contact)
    df_left_sub = df_left[df_left['Contact'] == contact]
    df_right_sub = df_right[df_right['Contact'] == contact]
    contact_name = 'C' + str(contact) #contact labels on y axis
    #Left Side
    ax = fig.add_subplot(4,2,location[contact]-1)
    ax = sns.barplot(x="Amplitude", y="Percent_Activation", hue="Fiber_Name", palette=colors, data=df_left_sub)
    ax.set(ylim=(0, 100))
    ax.set_ylabel(contact_name,fontsize=25,rotation=0,fontweight='bold')
    ax.yaxis.set_label_coords(-0.05,0.45)
    ax.set_xlabel('')
    ax.set_yticks([])
    if contact != 0:
        ax.set_xticks([])
    else:
        ax.set_xticklabels([-1,-2,-3,-4,-5],fontsize=25) #amplitude values
        ax.set_xlabel('Amplitude (mA)',fontsize=20,fontweight='bold')
    if contact == 3:
        ax.set_title('LEFT',fontsize=25,y=1.2,fontweight='bold')
        ax.legend(bbox_to_anchor=(1.6,-0.5),prop={'size':25}) #legend formatting, only 1 is needed
    else:
        ax.legend_.remove()
    
    #Right Side
    ax = fig.add_subplot(4,2,location[contact])
    ax = sns.barplot(x="Amplitude", y="Percent_Activation", hue="Fiber_Name", palette=colors, data=df_right_sub)
    ax.set(ylim=(0, 100))
    ax.legend_.remove()
    ax.set_ylabel(contact_name,fontsize=25,rotation=0,fontweight='bold')  
    ax.yaxis.set_label_coords(-0.05,0.45)
    ax.set_xlabel('')
    ax.set_yticks([])
    if contact != 0:
        ax.set_xticks([])
    else:
        ax.set_xticklabels([-1,-2,-3,-4,-5],fontsize=25)
        ax.set_xlabel('Amplitude (mA)',fontsize=20,fontweight='bold')
    if contact == 3:
        ax.set_title('RIGHT',fontsize=25,y=1.2,fontweight='bold')

plt.subplots_adjust(wspace=0.7) #give more space between the plots   
fig.text(0.06,0.5,'Percent Activation',va = 'center',rotation='vertical',fontsize=25,fontweight='bold')
plt.show()